In [1]:
import os
import time
import math
import random
import pickle
import numpy as np
import tensorflow as tf
import multiprocess as mp
from libs.TicTacToe import TicTacToe
from libs.model import create_az_model
from libs.MCTS import MCTS
from libs.Trainer import Trainer

In [2]:
n_cores = mp.cpu_count()
print(f"[Number of cores]: {n_cores}")

[Number of cores]: 16


## Tic-Tac-Toe engine

In [3]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])
TicTacToe().get_available_actions(state)

array([1., 1., 0., 0., 0., 0., 0., 0., 1.])

In [4]:
ttt = TicTacToe()

while ttt.status=="Ongoing":

    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)
    print(np.reshape(ttt.board, (3,3)),"\n")
    if len(valid_moves)==0:
        break

print("[Winner]:",ttt.winner)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]] 

[[ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]] 

[[ 0.  0.  0.]
 [-1.  0.  1.]
 [ 0.  0.  1.]] 

[[ 0.  0.  0.]
 [-1.  0.  1.]
 [ 0. -1.  1.]] 

[[ 1.  0.  0.]
 [-1.  0.  1.]
 [ 0. -1.  1.]] 

[[ 1.  0. -1.]
 [-1.  0.  1.]
 [ 0. -1.  1.]] 

[[ 1.  0. -1.]
 [-1.  0.  1.]
 [ 1. -1.  1.]] 

[[ 1.  0. -1.]
 [-1. -1.  1.]
 [ 1. -1.  1.]] 

[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1. -1.  1.]] 

[Winner]: 0


## Create policy and probability model

In [5]:
model = create_az_model()

In [6]:
model.summary()

Model: "tictactoe_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 9)]                  0         []                            
                                                                                                  
 tf.__operators__.add (TFOp  (None, 9)                    0         ['input_1[0][0]']             
 Lambda)                                                                                          
                                                                                                  
 tf.cast (TFOpLambda)        (None, 9)                    0         ['tf.__operators__.add[0][0]']
                                                                                                  
 tf.one_hot (TFOpLambda)     (None, 9, 3)                 0         ['tf.cast[0][0]'

In [7]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Monte Carlo Tree Search

In [8]:
mcts = MCTS(game=TicTacToe(), n_simulations=100)

In [9]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])

In [10]:
root = mcts.run(model=model, state=state, player=1)

1/1 [==============================] - 0s 16ms/step


In [11]:
root

State:
[ 0.  0. -1. -1.  1. -1.  1.  1.  0.]
Player:1
Value:0.96
Leaf:False
Visits:100
Children:
 -0: prior=0.3333333333333333
 -1: prior=0.3333333333333333
 -8: prior=0.3333333333333333

# Generating the first dataset

In [12]:
STARTING_TEMPERATURE = 10.0

In [13]:
# speed test on 100 games
start_time = time.time()
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
dataset = trainer.create_dataset(number_of_games=100, temperature=starting_temperature)
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 15.1300208568573 seconds


In [14]:
def parallel_fn(n):
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.MCTS import MCTS
    N_GAMES_PER_JOB = 25
    starting_temperature = 10.0
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE)
    return game_batch

In [15]:
start_time = time.time()
ctx = mp.get_context("spawn")
pool = ctx.Pool(n_cores)
dataset = pool.map(parallel_fn, range(n_cores))
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 17.62233281135559 seconds


In [16]:
N_GAMES_PER_JOB = 25
filename = str(N_GAMES_PER_JOB*n_cores)+"_tictactoe_temperature_"+str(starting_temperature)+".pickle"

flatten_dataset = []
for batch in dataset:
    flatten_dataset += batch
    
with open(filename, "wb") as fp:
    pickle.dump(flatten_dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)

# 1st training

In [17]:
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=create_az_model())

with open(filename, 'rb') as handle:
    dataset = pickle.load(handle)

In [18]:
fit_history = trainer.train(flatten_dataset, n_epochs=1000, batch_size=30, learning_rate=1e-2)

Epoch 1/1000


90/90 [==============================] - 2s 7ms/step - loss: 3.5886 - policy_loss: 1.8836 - value_loss: 1.6082 - val_loss: 3.0477 - val_policy_loss: 2.1091 - val_value_loss: 0.8242 - lr: 0.0100
Epoch 2/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.6339 - policy_loss: 1.6385 - value_loss: 0.8821 - val_loss: 3.0722 - val_policy_loss: 2.1497 - val_value_loss: 0.8138 - lr: 0.0100
Epoch 3/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.4557 - policy_loss: 1.5156 - value_loss: 0.8340 - val_loss: 2.5625 - val_policy_loss: 1.6481 - val_value_loss: 0.8123 - lr: 0.0100
Epoch 4/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.4074 - policy_loss: 1.4710 - value_loss: 0.8350 - val_loss: 2.4468 - val_policy_loss: 1.5361 - val_value_loss: 0.8123 - lr: 0.0100
Epoch 5/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.3708 - policy_loss: 1.4393 - value_loss: 0.8336 - val_loss: 2.3364 - val_policy_loss: 1.4290 -

90/90 [==============================] - 0s 4ms/step - loss: 2.1754 - policy_loss: 1.2720 - value_loss: 0.8324 - val_loss: 2.1686 - val_policy_loss: 1.2854 - val_value_loss: 0.8123 - lr: 2.2823e-04
Epoch 39/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1776 - policy_loss: 1.2744 - value_loss: 0.8324 - val_loss: 2.1678 - val_policy_loss: 1.2848 - val_value_loss: 0.8123 - lr: 1.7249e-04
Epoch 40/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1761 - policy_loss: 1.2732 - value_loss: 0.8324 - val_loss: 2.1679 - val_policy_loss: 1.2851 - val_value_loss: 0.8123 - lr: 1.2907e-04
Epoch 41/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1758 - policy_loss: 1.2730 - value_loss: 0.8324 - val_loss: 2.1681 - val_policy_loss: 1.2854 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 42/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1735 - policy_loss: 1.2708 - value_loss: 0.8324 - val_loss: 2.1679 - val_policy_loss: 1.2

Epoch 77/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1643 - policy_loss: 1.2672 - value_loss: 0.8324 - val_loss: 2.1634 - val_policy_loss: 1.2865 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 78/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1641 - policy_loss: 1.2672 - value_loss: 0.8324 - val_loss: 2.1632 - val_policy_loss: 1.2864 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 79/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1654 - policy_loss: 1.2686 - value_loss: 0.8324 - val_loss: 2.1631 - val_policy_loss: 1.2865 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 80/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1619 - policy_loss: 1.2652 - value_loss: 0.8324 - val_loss: 2.1630 - val_policy_loss: 1.2865 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 81/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1611 - policy_loss: 1.2646 - value_loss: 0.8324 - val_loss: 2.1622 - val_po

Epoch 116/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1533 - policy_loss: 1.2621 - value_loss: 0.8324 - val_loss: 2.1571 - val_policy_loss: 1.2861 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 117/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1554 - policy_loss: 1.2643 - value_loss: 0.8324 - val_loss: 2.1572 - val_policy_loss: 1.2863 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 118/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1538 - policy_loss: 1.2629 - value_loss: 0.8324 - val_loss: 2.1567 - val_policy_loss: 1.2859 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 119/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1541 - policy_loss: 1.2634 - value_loss: 0.8324 - val_loss: 2.1570 - val_policy_loss: 1.2864 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 120/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1547 - policy_loss: 1.2641 - value_loss: 0.8324 - val_loss: 2.1570 - v

90/90 [==============================] - 0s 5ms/step - loss: 2.1461 - policy_loss: 1.2600 - value_loss: 0.8324 - val_loss: 2.1528 - val_policy_loss: 1.2868 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 155/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1470 - policy_loss: 1.2609 - value_loss: 0.8324 - val_loss: 2.1524 - val_policy_loss: 1.2865 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 156/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1466 - policy_loss: 1.2607 - value_loss: 0.8324 - val_loss: 2.1521 - val_policy_loss: 1.2863 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 157/1000
90/90 [==============================] - 0s 4ms/step - loss: 2.1492 - policy_loss: 1.2634 - value_loss: 0.8324 - val_loss: 2.1523 - val_policy_loss: 1.2866 - val_value_loss: 0.8123 - lr: 1.0000e-04
Epoch 158/1000
90/90 [==============================] - 0s 5ms/step - loss: 2.1476 - policy_loss: 1.2619 - value_loss: 0.8324 - val_loss: 2.1522 - val_policy_loss:

In [19]:
trainer.model.predict(np.array([[0., 0., -1., -1., 1., -1., 1., 1., 0.]]))

1/1 [==============================] - 0s 129ms/step


(array([[1.5292804e-02, 3.7834185e-01, 6.2949868e-04, 2.4268795e-05,
         1.5596172e-04, 8.0116923e-05, 1.5714881e-04, 6.8897156e-05,
         6.0524940e-01]], dtype=float32),
 array([[0.07633218]], dtype=float32))

In [20]:
trainer.model.predict(np.array([[0., 0., 0., 0., -1., 1., -1., 0., 0.]]))

1/1 [==============================] - 0s 24ms/step


(array([[7.0850857e-02, 8.1257693e-02, 5.1002884e-01, 1.2070657e-01,
         2.4216896e-04, 4.6085939e-04, 1.1106054e-03, 9.6659914e-02,
         1.1868248e-01]], dtype=float32),
 array([[0.07633218]], dtype=float32))

In [21]:
trainer.save_model('saved_model/tmp_model')

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


# Generating the self-playing dataset (2nd)

loaded_model = tf.keras.models.load_model("saved_model/alpha_zero_model_first_training_validation/")

# speed test on 10 games
start_time = time.time()
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=loaded_model)
dataset = trainer.create_dataset(number_of_games=10, temperature=1)
print("Running time: %s seconds" % (time.time() - start_time))

In [ ]:
# 1175s -> ~30m for 100
# 25*0.5 ~ 12hs

In [ ]:
model = create_az_model()

In [22]:
N_ROUNDS = 40
N_GAMES_PER_JOB = 25

In [23]:
def parallel_fn(job_n, n):
    
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.MCTS import MCTS
    import tensorflow as tf
    N_GAMES_PER_JOB = 25
    STARTING_TEMPERATURE = 10.0
    
    model = tf.keras.models.load_model("saved_model/tmp_model")
    
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=model)
    
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE/n)
    
    return game_batch

#stored previous self-plays
filename_data = "4000_tictactoe_temperature_1_2nd_round.pickle"
if os.path.isfile(filename_data):
    with open(filename_data, 'rb') as handle:
        self_play_dataset = pickle.load(handle)
    print(f"Reusing {filename_data}")
else:
    self_play_dataset = []
    print(f"Starting from empty self-play data")

model_tmp_filename = "saved_model/tmp_model"
if os.path.isdir(model_tmp_filename):
    model = tf.keras.models.load_model(model_tmp_filename)
    print(f"Starting with {model_tmp_filename}")
else:
    model = tf.keras.models.load_model("saved_model/alpha_zero_model_first_training_validation/")
    print(f"Starting with saved_model/alpha_zero_model_first_training_validation/")

model.save(model_tmp_filename)

In [ ]:
self_play_dataset = flatten_dataset

for n in range(N_ROUNDS):
    
    start_time = time.time()
    
    ctx = mp.get_context("spawn")
    
    pool = ctx.Pool(n_cores)
    
    args = list(zip(range(n_cores),[n+1]*n_cores))
    
    round_dataset = pool.starmap(parallel_fn, args)
    print(f"Running time {n+1}-th round: {time.time() - start_time} seconds")
    
    flatten_round_dataset = []
    for batch in round_dataset:
        flatten_round_dataset += batch
    self_play_dataset += flatten_round_dataset
    
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=model)
    fit_history = trainer.train(self_play_dataset, n_epochs=50, batch_size=50, learning_rate=1e-2)
    
    trainer.model.save("saved_model/tmp_model")
    
    with open("selfplay_data.pickle", "wb") as fp:
        pickle.dump(self_play_dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)
    

Running time 1-th round: 784.0851500034332 seconds
Epoch 1/50
99/99 [==============================] - 2s 7ms/step - loss: 3.4331 - policy_loss: 1.8701 - value_loss: 1.4685 - val_loss: 2.8795 - val_policy_loss: 1.8861 - val_value_loss: 0.8879 - lr: 0.0100
Epoch 2/50
99/99 [==============================] - 1s 5ms/step - loss: 2.5445 - policy_loss: 1.5845 - value_loss: 0.8573 - val_loss: 2.7668 - val_policy_loss: 1.7774 - val_value_loss: 0.8885 - lr: 0.0100
Epoch 3/50
99/99 [==============================] - 0s 5ms/step - loss: 2.4404 - policy_loss: 1.4814 - value_loss: 0.8578 - val_loss: 2.4676 - val_policy_loss: 1.4782 - val_value_loss: 0.8879 - lr: 0.0100
Epoch 4/50
99/99 [==============================] - 0s 5ms/step - loss: 2.3856 - policy_loss: 1.4258 - value_loss: 0.8581 - val_loss: 2.4274 - val_policy_loss: 1.4353 - val_value_loss: 0.8896 - lr: 0.0100
Epoch 5/50
99/99 [==============================] - 0s 5ms/step - loss: 2.3574 - policy_loss: 1.3974 - value_loss: 0.8575 - val_l

99/99 [==============================] - 0s 5ms/step - loss: 2.1413 - policy_loss: 1.2390 - value_loss: 0.8565 - val_loss: 2.1768 - val_policy_loss: 1.2427 - val_value_loss: 0.8885 - lr: 1.2907e-04
Epoch 41/50
99/99 [==============================] - 1s 5ms/step - loss: 2.1403 - policy_loss: 1.2383 - value_loss: 0.8565 - val_loss: 2.1765 - val_policy_loss: 1.2426 - val_value_loss: 0.8885 - lr: 1.0000e-04
Epoch 42/50
99/99 [==============================] - 0s 5ms/step - loss: 2.1408 - policy_loss: 1.2389 - value_loss: 0.8565 - val_loss: 2.1767 - val_policy_loss: 1.2430 - val_value_loss: 0.8884 - lr: 1.0000e-04
Epoch 43/50
99/99 [==============================] - 1s 6ms/step - loss: 2.1406 - policy_loss: 1.2389 - value_loss: 0.8565 - val_loss: 2.1770 - val_policy_loss: 1.2435 - val_value_loss: 0.8884 - lr: 1.0000e-04
Epoch 44/50
99/99 [==============================] - 0s 5ms/step - loss: 2.1407 - policy_loss: 1.2392 - value_loss: 0.8565 - val_loss: 2.1760 - val_policy_loss: 1.2427 - va

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


Running time 2-th round: 699.984493970871 seconds
Epoch 1/50
144/144 [==============================] - 2s 6ms/step - loss: 2.3448 - policy_loss: 1.4013 - value_loss: 0.8670 - val_loss: 2.3434 - val_policy_loss: 1.3293 - val_value_loss: 0.9194 - lr: 0.0100
Epoch 2/50
144/144 [==============================] - 1s 5ms/step - loss: 2.3313 - policy_loss: 1.3598 - value_loss: 0.8680 - val_loss: 2.3308 - val_policy_loss: 1.3047 - val_value_loss: 0.9188 - lr: 0.0100
Epoch 3/50
144/144 [==============================] - 1s 5ms/step - loss: 2.3259 - policy_loss: 1.3467 - value_loss: 0.8681 - val_loss: 2.3273 - val_policy_loss: 1.2967 - val_value_loss: 0.9191 - lr: 0.0100
Epoch 4/50
144/144 [==============================] - 1s 5ms/step - loss: 2.3148 - policy_loss: 1.3349 - value_loss: 0.8686 - val_loss: 2.3572 - val_policy_loss: 1.3267 - val_value_loss: 0.9202 - lr: 0.0100
Epoch 5/50
144/144 [==============================] - 1s 5ms/step - loss: 2.3227 - policy_loss: 1.3388 - value_loss: 0.867

144/144 [==============================] - 1s 5ms/step - loss: 2.1372 - policy_loss: 1.2250 - value_loss: 0.8671 - val_loss: 2.1308 - val_policy_loss: 1.1666 - val_value_loss: 0.9191 - lr: 1.2907e-04
Epoch 41/50
144/144 [==============================] - 1s 5ms/step - loss: 2.1362 - policy_loss: 1.2241 - value_loss: 0.8671 - val_loss: 2.1302 - val_policy_loss: 1.1662 - val_value_loss: 0.9191 - lr: 1.0000e-04
Epoch 42/50
144/144 [==============================] - 1s 5ms/step - loss: 2.1367 - policy_loss: 1.2248 - value_loss: 0.8671 - val_loss: 2.1301 - val_policy_loss: 1.1662 - val_value_loss: 0.9191 - lr: 1.0000e-04
Epoch 43/50
144/144 [==============================] - 1s 5ms/step - loss: 2.1363 - policy_loss: 1.2246 - value_loss: 0.8671 - val_loss: 2.1286 - val_policy_loss: 1.1649 - val_value_loss: 0.9191 - lr: 1.0000e-04
Epoch 44/50
144/144 [==============================] - 1s 5ms/step - loss: 2.1366 - policy_loss: 1.2250 - value_loss: 0.8671 - val_loss: 2.1290 - val_policy_loss: 1

...

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


Running time 31-th round: 608.1484541893005 seconds
Epoch 1/50
1854/1854 [==============================] - 14s 7ms/step - loss: 1.3006 - policy_loss: 0.9588 - value_loss: 0.2974 - val_loss: 0.9860 - val_policy_loss: 0.9231 - val_value_loss: 0.0191 - lr: 0.0100
Epoch 2/50
1854/1854 [==============================] - 12s 7ms/step - loss: 1.2995 - policy_loss: 0.9578 - value_loss: 0.2974 - val_loss: 0.8503 - val_policy_loss: 0.7876 - val_value_loss: 0.0184 - lr: 0.0100
Epoch 3/50
1854/1854 [==============================] - 12s 7ms/step - loss: 1.3007 - policy_loss: 0.9584 - value_loss: 0.2973 - val_loss: 0.8425 - val_policy_loss: 0.7785 - val_value_loss: 0.0193 - lr: 0.0100
Epoch 4/50
1854/1854 [==============================] - 12s 7ms/step - loss: 1.3004 - policy_loss: 0.9580 - value_loss: 0.2974 - val_loss: 0.8445 - val_policy_loss: 0.7808 - val_value_loss: 0.0181 - lr: 0.0100
Epoch 5/50
1854/1854 [==============================] - 12s 6ms/step - loss: 1.3016 - policy_loss: 0.9594 - 

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


Running time 32-th round: 653.1218447685242 seconds
Epoch 1/50
1919/1919 [==============================] - 15s 7ms/step - loss: 1.2843 - policy_loss: 0.9526 - value_loss: 0.2880 - val_loss: 1.1257 - val_policy_loss: 1.0655 - val_value_loss: 0.0161 - lr: 0.0100
Epoch 2/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2870 - policy_loss: 0.9542 - value_loss: 0.2881 - val_loss: 0.8762 - val_policy_loss: 0.8162 - val_value_loss: 0.0146 - lr: 0.0100
Epoch 3/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2864 - policy_loss: 0.9536 - value_loss: 0.2881 - val_loss: 0.8451 - val_policy_loss: 0.7862 - val_value_loss: 0.0146 - lr: 0.0100
Epoch 4/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2875 - policy_loss: 0.9546 - value_loss: 0.2881 - val_loss: 0.9917 - val_policy_loss: 0.9285 - val_value_loss: 0.0180 - lr: 0.0100
Epoch 5/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2873 - policy_loss: 0.9543 - 

1919/1919 [==============================] - 13s 7ms/step - loss: 1.2052 - policy_loss: 0.9055 - value_loss: 0.2877 - val_loss: 0.7955 - val_policy_loss: 0.7679 - val_value_loss: 0.0157 - lr: 1.7249e-04
Epoch 40/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2045 - policy_loss: 0.9050 - value_loss: 0.2877 - val_loss: 0.7952 - val_policy_loss: 0.7678 - val_value_loss: 0.0156 - lr: 1.2907e-04
Epoch 41/50
1919/1919 [==============================] - 12s 6ms/step - loss: 1.2047 - policy_loss: 0.9052 - value_loss: 0.2877 - val_loss: 0.7954 - val_policy_loss: 0.7681 - val_value_loss: 0.0157 - lr: 1.0000e-04
Epoch 42/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2045 - policy_loss: 0.9052 - value_loss: 0.2877 - val_loss: 0.7952 - val_policy_loss: 0.7679 - val_value_loss: 0.0158 - lr: 1.0000e-04
Epoch 43/50
1919/1919 [==============================] - 13s 7ms/step - loss: 1.2038 - policy_loss: 0.9046 - value_loss: 0.2877 - val_loss: 0.7952 - val